# iDOM - A Brief Introduction

iDOM is an easily installable and powerful Python package for:

- Making responsive user interfaces with pure Python

- Doing so with a declarative decorator-based API

- And rendering those layouts on the web or in Jupyter Notebooks


# This Notebook

Will guide you though the basics of iDOM through several examples:

1. [Slideshows - Basic Interactivity](#Slideshows---Basic-Interactivity)

2. [Plotting with Matplotlib](#Plotting-with-Matplotlib)

4. [To Do Lists - More on Interactivity](#To-Do-Lists---More-on-Interactivity)

3. [The Game Snake - Using What We've Learned](#The-Game-Snake---Using-What-We've-Learned)


## Where Can You Learn More?

Go to https://github.com/rmorshea/idom for:

- more documentation

- installation instructions

- trouble shooting support


In [1]:
import idom

### A Little Set Up

Since you may be running this notebook on [mybinder.org](https://mybinder.org) we have to do a little bit of work to make sure that your frontend view connects to the right URL. You shouldn't have to change anything here. Be sure to [post an issue](https://github.com/rmorshea/idom/issues) if you experience any problems. 

In [2]:
from example_utils import localhost, pretty_dict_string

webpage_url = localhost('http', 8765) + '/idom/client/index.html'
websocket_url = localhost('ws', 8765) + '/idom/stream'

show, element = idom.hotswap()
idom.SimpleServer(element).daemon("0.0.0.0", 8765)

<Thread(Thread-4, started 139672655804160)>

[2019-05-09 23:50:07 -0700] [8821] [INFO] Goin' Fast @ http://0.0.0.0:8765


# Slideshows - Basic Interactivity

Let's use iDOM to create a simple slideshow which changes whenever a user clicks an image. To do this we define a function called `Slideshow` which will return an `<img/>` element. Attached to this `<img/>` is a handler for `onClick` events created whenever a user clicks the image.

In [3]:
@idom.element
async def Slideshow(self, index=0):
    events = idom.Events()

    @events.on("Click")
    async def change():
        self.update(index + 1)

    url = f"https://picsum.photos/800/300?image={index}"
    return idom.node("img", src=url, eventHandlers=events)

## Try it Out!

The following cells will display the output here. However you can also see the same thing in a standard web page:

In [4]:
print('Get a webpage that streams layout updates via a websocket:')
print(webpage_url)

Get a webpage that streams layout updates via a websocket:
http://127.0.0.1:8765/idom/client/index.html


If you just want to see it here run the cell below...

In [5]:
print("Try clicking the image! 🖱️")

show(Slideshow)

idom.display("jupyter", websocket_url)

Try clicking the image! 🖱️


JupyterWigdet('ws://127.0.0.1:8765/idom/stream')

## What Just Happened?

At the core of iDOM are simple dictionary structures that model an HTML layout.

To help you understand what's happening let's see what this dictionary model looks like:

In [6]:
slideshow = Slideshow()
model = await slideshow.render()
print(pretty_dict_string(model))

{
 'tagName': 'img',
 'eventHandlers': {
   'onClick': '37dab1416d914803a96977fead074506_onClick',
  },
 'attributes': {
   'src': 'https://picsum.photos/800/300?image=0',
  },
}


This will render the following HTML

```html
<img src='https://picsum.photos/800/300?image={index}' onClick={slideshow-change-callback}/>
```

Where each time the user clicks the rendered image:

1. The `slideshow-change-callback` is triggered and calls `change()`

2. The `change()` function then tells the `Slideshow` element to update with a new `{index}`.

3. A new image is rendered for the user with the updated `src` index.

# Plotting with Matplotlib

The following example shows how you might create an animated matplotlib plot of 1D [Brownian Motion](https://en.wikipedia.org/wiki/Brownian_motion). Matplotlib provides similar functionality natively, though with the limitation that all the frames of the animation are pre-computed. At the moment `idom` doesn't have native support for matplotlib, by you can still render them by:

1. Creating an `Image` element with some format (e.g. SVG).

2. Calling a figure's `savefig` method and writing to the `Image.io` attribute.

In [7]:
%matplotlib agg
from matplotlib import pyplot as plt

## New Feature - Animation Callback

The animation callback is a special hook registered by decorating a function with `self.animate` upon rendering an element. This hook is called repeatedly each time a render takes place allowing you to update a view without having to wait for user interaction.

In the case of our plot we use it to step through

In [41]:
import time
import asyncio
import random


@idom.element
async def RandomWalk(self):
    x, y = [0] * 50, [0] * 50
    plot = Plot(x, y)

    @self.animate(rate=0.3)
    async def walk():
        x.pop(0)
        x.append(x[-1] + 1)
        y.pop(0)
        y.append(y[-1] + random.gauss(0, 0.5))
        plot.update(x, y)

    return plot

@idom.element
async def Plot(self, x, y):
    fig, axes = plt.subplots()
    axes.plot(x, y)
    img = idom.Image("svg")
    fig.savefig(img.io, format="svg")
    plt.close(fig)
    return img

## Try it Out

In [42]:
print("Try clicking the plot! 📈")

show(RandomWalk)

idom.display("jupyter", websocket_url)

Try clicking the plot! 📈


JupyterWigdet('ws://127.0.0.1:8765/idom/stream')

# To Do Lists - More on Interactivity

So far we've created event handlers which respond to simpler events like clicks, but often we want to track more complicated events. In this example we'll create a to-do list. To accomplish this goal we'll need to accept user inputs and track key presses. Let's take a look at the `Events` API in more depths to learn how. Consider this generalized definition of an event handler:

```python
events = idom.Events()

@events.on("EventName", using="y: path.to.value")
def handler(x, y):
    # respond to the event somehow
    ....
```

Here we see the decorator syntax for defining events we're familiar with, however there are some differences:

1. **The `handler` accepts arguments `x, y`**

The arguments of the `handler` function are used to refer to attributes of [event objects](https://reactjs.org/docs/events.html#supported-events) produced by user interactions. For example consider [keyboard events](https://reactjs.org/docs/events.html#keyboard-events) they have attributes like `key` which is the name of the current key being pressed (e.g. "ArrowLeft", "h", or "2").

2. **`events.on` defines `using` with a string that refers to an argument from `handler`**

In a minority of cases you may need to access to information that's nested inside one of the event's own attributes. In the case of our to-do list we have an `<input/>` element and each time the user changes it we'd like to know what its contents are. Unfortunately the `onKeyDown` event has no attribute that can tell us this. Instead we need the current `value` of the event's `target`. To access this we declare:

```python
@event.on("KeyDown", using="value=target.value")
def handler(value):
    ...
```

Which will pass the `target.value` to the `value` argument of our `handler`


## The Code

In [10]:
@idom.element
async def Todo(self):
    items = []

    task_input = idom.html.input(eventHandlers=idom.Events())
    task_list = TaskList(items)

    @task_input.eventHandlers.on("KeyDown", using="value=target.value")
    async def add_new_task(key, value):
        if key == "Enter":
            items.append(value)
            task_list.update(items)

    return idom.html.div(task_input, task_list)


@idom.element
async def TaskList(self, items):
    tasks = []

    for text in items:
        task_text = idom.html.td(idom.html.p(text))
        delete_button = idom.html.td(idom.html.button("x"), eventHandlers=idom.Events())

        @delete_button.eventHandlers.on("Click")
        async def remove(text=text):
            for i, t in enumerate(items):
                if t is text:
                    del items[i]
                    break
            self.update(items)

        tasks.append(idom.html.tr(task_text, delete_button))

    return idom.node("table", tasks)

## Try it Out

In [11]:
print("Try typing in the text box and pressing 'Enter' 📋")

show(Todo)

idom.display("jupyter", websocket_url)

Try typing in the text box and pressing 'Enter' 📋


JupyterWigdet('ws://127.0.0.1:8765/idom/stream')

# The Game Snake - Using What We've Learned

To show off what's possible with iDOM, let's look at how to make the game Snake.


## New Feature - Element State

What's new here is the declaration of `state` when creating an `@element` function:

```python
@idom.element(state="x, y")
def MyElement(self, x, y):
    ...
```

Each parameter of `MyElement` specified in `state` will have its value preserved across updates unless explicitely changed. So for example if we instantiated `MyElement(x=1, y=1)` and later called `self.update()` both `x` and `y` would be passed in as `1` again. However calling `self.update(y=2)` would cause `MyElement` to render with `x=1, y=2` instead. Calling `self.update(x=3)` once would have `MyElement` render with `x=3, y=2`.

In our `Game` element we use this to preserve the original `grid_size` and `block_size` after the player loses and we need to reset the game.

In [12]:
import enum
import time
import random
import asyncio


class WASD(enum.Enum):
    w = (-1, 0)
    a = (0, -1)
    s = (1, 0)
    d = (0, 1)


@idom.element(state="grid_size, block_size")
async def Game(self, grid_size, block_size):
    game = idom.Bunch(
        snake=[],
        grid=Grid(grid_size, block_size),
        new_direction=idom.Var(WASD.d),
        old_direction=idom.Var(WASD.d),
        food=idom.Var(None),
        won=idom.Var(False),
        lost=idom.Var(False),
    )

    @game.grid.eventHandlers.on("KeyDown")
    async def direction_change(key):
        if hasattr(WASD, key):
            game.new_direction.set(WASD[key])

    game.snake.extend(
        [
            (grid_size // 2 - 1, grid_size // 2 - 3),
            (grid_size // 2 - 1, grid_size // 2 - 2),
            (grid_size // 2 - 1, grid_size // 2 - 1),
        ]
    )

    grid_points = set((x, y) for x in range(grid_size) for y in range(grid_size))

    def set_new_food():
        points_not_in_snake = grid_points.difference(game.snake)
        new_food = random.choice(list(points_not_in_snake))
        get_grid_block(game.grid, new_food).update("blue")
        game.food.set(new_food)

    @self.animate(rate=0.5)
    async def loop():
        if game.won.get() or game.lost.get():
            self.update()
            await asyncio.sleep(1)
            return False
        else:
            await draw(game, grid_size, set_new_food)

    set_new_food()
    return game.grid


async def draw(game, grid_size, set_new_food):
    if game.snake[-1] in game.snake[:-1]:
        # point out where you touched
        get_grid_block(game.grid, game.snake[-1]).update("red")
        game.lost.set(True)
        return

    vector_sum = tuple(
        map(sum, zip(game.old_direction.get().value, game.new_direction.get().value))
    )
    if vector_sum != (0, 0):
        game.old_direction.set(game.new_direction.get())

    new_head = (
        # grid wraps due to mod op here
        (game.snake[-1][0] + game.old_direction.get().value[0]) % grid_size,
        (game.snake[-1][1] + game.old_direction.get().value[1]) % grid_size,
    )

    game.snake.append(new_head)

    if new_head == game.food.get():
        if len(game.snake) == grid_size * grid_size:
            game.won.set(True)
            return
        set_new_food()
    else:
        get_grid_block(game.grid, game.snake.pop(0)).update("white")

    # update head after tail - new head may be the same as the old tail
    get_grid_block(game.grid, new_head).update("black")


def Grid(grid_size, block_size):
    return idom.html.div(
        [
            idom.html.div(
                [Block("white", block_size) for i in range(grid_size)],
                style={"height": block_size},
            )
            for i in range(grid_size)
        ],
        style={
            "height": f"{block_size * grid_size}px",
            "width": f"{block_size * grid_size}px",
        },
        eventHandlers=idom.Events(),
        tabIndex=-1,
    )


@idom.element(state="block_size")
async def Block(self, color, block_size):
    return idom.html.div(
        style={
            "height": f"{block_size}px",
            "width": f"{block_size}px",
            "backgroundColor": color,
            "display": "inline-block",
            "border": "1px solid white",
        }
    )


def get_grid_block(grid, point):
    x, y = point
    return grid["children"][x]["children"][y]


In [13]:
print("Click to start playing 🎮")
print()
print("Slow internet may cause inconsistent frame pacing 😅")

show(Game, 10, 40)

idom.display("jupyter", websocket_url)

Click to start playing 🎮

Slow internet may cause inconsistent frame pacing 😅


JupyterWigdet('ws://127.0.0.1:8765/idom/stream')